# Multivariate Linear Regression, Non-Parametric Models and Cross-Validation

**Author**: Christo Pananjickal Baby

**Course**: Applied Artificial Intelligence and Machine Learning

**Assignment**: Practical Lab 2

**Date**: 18-06-2025

Kindly run the code blocks in order or click run all.